# Batchfitting of multiple datasets

In [ ]:
%matplotlib notebook
import matplotlib.pyplot as plt
from multiprocessing import Pool

In [ ]:
from copy import deepcopy
import glob
from natsort import natsorted

import numpy as np
import refnx
from refnx.dataset import Data1D
from refnx.analysis import Objective, CurveFitter
from refnx.reflect import SLD, Slab, ReflectModel, Structure
from refnx.dataset import ReflectDataset

print(refnx.version.version)

# natsort is used for alphanumeric sort
files = natsorted(list(glob.iglob('./*.dat')))

"""
initial model setup
"""
# set up the SLD objects for each layer
sld0 = SLD(0.0 + 0.0j, name='')
sld1 = SLD(6.14127029941648 + 0.0j, name='')
sld2 = SLD(3.47 + 0.0j, name='')
sld3 = SLD(2.07 + 0.0j, name='')

# set up the Slab objects for each layer
slab0 = Slab(0.0, sld0, 0.0, name='')
slab1 = Slab(520.5277261491321, sld1, 8.762992087388763, name='')
slab2 = Slab(15.305814908968332, sld2, 5.020239736927396, name='')
slab3 = Slab(0.0, sld3, 3.0, name='')

# set up the limits for SLD's and Slabs
sld1.real.setp(vary=True, bounds=(0, 7.0))
slab1.thick.setp(vary=True, bounds=(200, 1020.0))
slab1.rough.setp(vary=True, bounds=(2.0, 20.0))
slab2.thick.setp(vary=True, bounds=(5.0, 10.0))
slab2.rough.setp(vary=True, bounds=(2.0, 10.0))

# set up the Structure object from the Slabs
structure = slab0 | slab1 | slab2 | slab3

# make the reflectometry model
model = ReflectModel(structure, scale=1.0, bkg=1e-10, dq=8.0)
model.scale.setp(vary=True, bounds=(0.5, 1.5))
model.bkg.setp(vary=True, bounds=(1e-10, 1e-5))

"""
Now do the fit
"""
filenames = []
models = []
objectives = []

for idx, file in enumerate(files):
    data = Data1D(data=file)

    # make the objective
    objective = Objective(deepcopy(model), data)

    filenames.append(data.filename)
    models.append(objective.model)
    objectives.append(objective)

In [ ]:
# lets do the analysis in parallel. The order shouldn't
# matter because we use differential evolution
def fun(objective):
    # make the curvefitter and do the fit
    fitter = CurveFitter(objective)
    fitter.fit('differential_evolution')
    return objective

In [ ]:
with Pool(4) as p:
    l = list(p.map(fun, objectives))

In [ ]:
thickness = []
sld = []
for objective in l:
    model = objective.model
    thickness.append(model.structure[1].thick.value)
    sld.append(model.structure[1].sld.real.value)

fig, ax1 = plt.subplots()
ax1.set_xlabel('dataset')
ax1.set_ylabel('thickness (A)')
ax1.plot(thickness, 'b')
ax2 = ax1.twinx()
ax2.plot(sld, 'r')
ax2.set_ylabel('SLD');